In [18]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [19]:
df_main = pd.read_csv('data/df_main.csv').drop(columns=["Unnamed: 0"])

In [20]:
df_main['product'].value_counts()

Electricity                                          21957
Total Combustible Fuels                               4128
Total Renewables (Hydro, Geo, Solar, Wind, Other)     4128
Combustible Renewables                                4109
Natural Gas                                           4043
Wind                                                  4036
Solar                                                 4032
Oil and Petroleum Products                            4019
Hydro                                                 4010
Coal, Peat and Manufactured Gases                     3743
Other Combustible Non-Renewables                      3732
Not Specified                                         2946
Nuclear                                               2737
Geothermal                                            2085
Other Renewables                                      1392
Name: product, dtype: int64

In [21]:
df_main.loc[df_main['product'] == 'Electricity']['country_name'].value_counts()

Austria            1008
Ireland            1008
Spain              1008
Slovenia           1008
Slovak Republic    1008
Portugal           1008
Poland             1008
Luxembourg         1008
Belgium            1008
Italy              1008
Lithuania          1008
Germany            1008
France             1008
Czech Republic     1008
Greece             1007
Sweden             1006
Latvia              960
Hungary             888
Netherlands         888
Finland             888
Estonia             888
Denmark             888
Bulgaria            108
Croatia             108
Cyprus              108
Malta               108
Name: country_name, dtype: int64

In [22]:
df_main.loc[(df_main['product'] == 'Electricity') & (df_main['country_name'] == 'Malta')]

,country_name,date,parameter,product,value,unit,year
396,Malta,12/1/2023,Net Electricity Production,Electricity,134,GWh,2023
799,Malta,11/1/2023,Net Electricity Production,Electricity,146,GWh,2023
1202,Malta,10/1/2023,Net Electricity Production,Electricity,168,GWh,2023
1606,Malta,9/1/2023,Net Electricity Production,Electricity,205,GWh,2023
2010,Malta,8/1/2023,Net Electricity Production,Electricity,225,GWh,2023
...,...,...,...,...,...,...,...
46791,Malta,5/1/2015,Net Electricity Production,Electricity,89,GWh,2015
47200,Malta,4/1/2015,Net Electricity Production,Electricity,102,GWh,2015
47610,Malta,3/1/2015,Net Electricity Production,Electricity,170,GWh,2015
48020,Malta,2/1/2015,Net Electricity Production,Electricity,169,GWh,2015


In [30]:
# Classify all EU countries according to the average annual global solar radiation [kWh/m2] (period 1994-2016)

eu_countries_solar_radiation_classification_dict = {
    'Austria': '2: low',
    'Belgium': '2: low',
    'Bulgaria': '3: medium',
    'Croatia': '3: medium',
    'Cyprus': '5: very high',
    'Czech Republic': '2: low',
    'Denmark': '1: very low',
    'Estonia': '2: low',
    'Finland': '1: very low',
    'France': '3: medium',
    'Germany': '2: low',
    'Greece': '4: high',
    'Hungary': '3: medium',
    'Ireland': '1: very low',
    'Italy': '4: high',
    'Latvia': '2: low',
    'Lithuania': '2: low',
    'Luxembourg': '2: low',
    'Malta': '5: very high',
    'Netherlands': '2: low',
    'Poland': '2: low',
    'Portugal': '4: high',
    'Romania': '3: medium',
    'Slovak Republic': '2: low',
    'Slovenia': '3: medium',
    'Spain': '5: very high',
    'Sweden': '1: very low'}

In [31]:
# Load demographic data (Population and Area)
df_pop = pd.read_csv("data/eu_country_population.csv")
df_area = pd.read_csv("data/eu_country_area.csv")

df_pop = df_pop.rename(columns={'OBS_VALUE': "population"})
df_area = df_area.rename(columns={'OBS_VALUE': "area_[km2]"})

df_pop = df_pop[df_pop['geo'] != "EU27_2020"]
df_area = df_area[df_area['geo'] != "EU27_2020"]

In [32]:
country_codes_dict = {
    'AT': 'Austria',
    'BE': 'Belgium',
    'BG': 'Bulgaria',
    'CY': 'Cyprus',
    'CZ': 'Czech Republic',
    'DE': 'Germany',
    'DK': 'Denmark',
    'EE': 'Estonia',
    'EL': 'Greece',
    'ES': 'Spain',
    'FI': 'Finland',
    'FR': 'France',
    'HR': 'Croatia',
    'HU': 'Hungary',
    'IE': 'Ireland',
    'IT': 'Italy',
    'LT': 'Lithuania',
    'LU': 'Luxembourg',
    'LV': 'Latvia',
    'MT': 'Malta',
    'NL': 'Netherlands',
    'PL': 'Poland',
    'PT': 'Portugal',
    'RO': 'Romania',
    'SE': 'Sweden',
    'SI': 'Slovenia',
    'SK': 'Slovak Republic'}

In [33]:
# Create dataframe with one row per country and aggregate demographic information
df_countries = df_pop[['geo', 'population']].merge(df_area[['geo', 'area_[km2]']], how='inner', on='geo')
df_countries = df_countries.rename(columns={'geo': 'country_code'})
df_countries['country_name'] = df_countries['country_code'].map(country_codes_dict)

# Add data about solar radiation classification
df_countries['solar_class'] = df_countries['country_name'].map(eu_countries_solar_radiation_classification_dict)

# Add data from df_main (solar electricity production and total electricity production)
df_pivot = df_main.loc[(df_main["product"] == "Solar") & (df_main["year"] > 2013)].pivot_table(index="country_name", values="value", aggfunc="mean")*365
df_countries = df_countries.merge(df_pivot, how='inner', on='country_name')
df_countries = df_countries.rename(columns={'value': 'solar_power_[GWh]'})

df_pivot = df_main.loc[(df_main["product"] == "Electricity") & (df_main["year"] > 2013)].pivot_table(index="country_name", values="value", aggfunc="mean")*365
df_countries = df_countries.merge(df_pivot, how='inner', on='country_name')
df_countries = df_countries.rename(columns={'value': 'total_annual_electricity_[GWh]'})


# Calculate solar power intensity (production per area)
df_countries['solar_power_intensity_[GWh_per_km2]'] = df_countries['solar_power_[GWh]'] / df_countries["area_[km2]"]
df_countries['solar_power_intensity_[GWh_per_1000_capita]'] = df_countries['solar_power_[GWh]'] / df_countries["population"] * 1000
df_countries['solar_share_in_total_electricity_[%]'] = df_countries['solar_power_[GWh]'] / df_countries["total_annual_electricity_[GWh]"] * 100

,solar_power_intensity_[GWh_per_km2]
solar_class,
1: very low,0.242875
2: low,1.666577
3: medium,0.480043
4: high,1.393859
5: very high,7.105441


In [28]:
df_countries

,country_code,population,area_[km2],country_name,solar_class,solar_power_[GWh],total_annual_electricity_[GWh],solar_power_intensity_[GWh_per_km2],solar_power_intensity_[GWh_per_1000_capita],solar_share_in_total_electricity_[%]
0,AT,9104772,83878,Austria,2: low,5.138288e+04,9.444436e+05,0.612591,5.643510,5.440545
1,BE,11742796,30667,Belgium,2: low,1.383837e+05,1.004391e+06,4.512462,11.784559,13.777871
2,BG,6447710,110996,Bulgaria,3: medium,5.135685e+04,1.262052e+06,0.462691,7.965131,4.069314
3,CY,920701,9253,Cyprus,5: very high,1.032815e+04,1.459088e+05,1.116195,11.217701,7.078498
4,CZ,10827529,78871,Czech Republic,2: low,6.904583e+04,9.266453e+05,0.875427,6.376878,7.451161
5,DE,84358845,357569,Germany,2: low,1.373805e+06,6.273871e+06,3.842070,16.285254,21.897249
6,DK,5932654,42925,Denmark,1: very low,3.816988e+04,5.306452e+05,0.889222,6.433862,7.193106
7,EE,1365884,45336,Estonia,2: low,5.455990e+03,1.526736e+05,0.120346,3.994475,3.573629
8,EL,10413982,131694,Greece,4: high,1.496378e+05,5.863504e+05,1.136254,14.368935,25.520206
9,ES,48085361,505983,Spain,5: very high,6.155025e+05,2.880798e+06,1.216449,12.800206,21.365696


In [50]:
fig = make_subplots(rows=3, cols=1, vertical_spacing=0.06,
                    subplot_titles=('Average Annual Solar Net Electricity Production',
                                    '',
                                    ''))

fig.add_trace(go.Bar(
                x=df_countries['country_code'],
                y=df_countries['solar_power_intensity_[GWh_per_km2]']),
                row=1, col=1
                )
fig.add_trace(go.Bar(
                x=df_countries['country_code'],
                y=df_countries['solar_power_intensity_[GWh_per_1000_capita]']),
                row=2, col=1
                )
fig.add_trace(go.Bar(
                x=df_countries['country_code'],
                y=df_countries['solar_share_in_total_electricity_[%]']),
                row=3, col=1
                )

fig.update_layout(height=800, width=800, showlegend=False)

fig.update_xaxes(title_text="Country Code", row=3, col=1)
fig.update_yaxes(title_text="GWh/km²", row=1, col=1)
fig.update_yaxes(title_text="GWh/1000 People", row=2, col=1)
fig.update_yaxes(title_text="Percent in total Electricity", row=3, col=1)

fig.show()

In [40]:
df_pivot_class = df_countries.pivot_table(index="solar_class", values=["solar_power_intensity_[GWh_per_km2]"], aggfunc="mean")

In [66]:
fig = px.bar(df_pivot_class, x=df_pivot_class.index, y="solar_power_intensity_[GWh_per_km2]",
             title='Average Annual Solar Net Electricity Production')

fig.update_layout(height=500, width=800, showlegend=False,
                  title={'y': 0.85, 'x': 0.5, 'xanchor': 'center', 'yanchor': 'top'})

fig.update_xaxes(title_text="Solar Radiation Category")
fig.update_yaxes(title_text="GWh/km²")

fig.show()